# **4. ABSA process with Open AI API**

In [4]:
# ===============================
# 1. Install + Setup
# ===============================
!pip install openai

import os
import json
import time
import pandas as pd
from openai import OpenAI

# Mount Google Drive (to save progress safely)
from google.colab import drive
drive.mount('/content/drive')

from google.colab import files

# Set API Key
from google.colab import userdata
api_key = userdata.get('openAI-API-ABSA')
client = OpenAI(api_key=api_key)

# ===============================
# 2. Load Data
# ===============================
input_path = '/content/drive/MyDrive/Colab Notebooks/ABSA-summer-project/trustpilot_lightyear_cleaned_review-only.csv'
df = pd.read_csv(input_path)

# Ensure unique ID
df = df.reset_index().rename(columns={"index": "review_id"})
df["review_id"] = df["review_id"] + 1
print("Total reviews:", len(df))

# ===============================
# 3. System Prompt (ABSA schema)
# ===============================

prompt = """
You are an expert in Aspect-Based Sentiment Analysis (ABSA) for customer reviews.
The company is **Lightyear**, a fintech investment app where users can invest in stocks and ETFs.
Your task is to analyze each review and extract mentioned aspects, their sentiment, and relevant details, using the fixed schema below.

### Locked Schema

| Code      | Category              | Aspect_normalized                          |
|-----------|-----------------------|--------------------------------------------|
| FTR_acc   | Features Account      | Account setup & funding/withdrawals        |
| FTR_acc   | Features Account      | Local tax-wrapper availability (e.g. TBSZ) |
| FTR_por   | Features Portfolio    | Portfolio overview & detail view           |
| FTR_por   | Features Portfolio    | Portfolio analytics & performance tracking |
| FTR_notif | Notifications & Alerts| Notifications, alerts, reminders           |
| INT_app   | Interface/Usability   | Mobile app usability                       |
| INT_web   | Interface/Usability   | Web interface usability                    |
| INT_srch  | Interface/Usability   | Search & filtering                         |
| INT_nav   | Interface/Usability   | Navigation, menus, layout                  |
| INT_des   | Interface/Usability   | Design & aesthetics                        |
| TRD_frac  | Trading Options       | Fractional shares/ETFs                     |
| TRD_ord   | Trading Options       | Order types (market, limit, stop, etc.)    |
| TRD_exec  | Trading Options       | Execution speed & quality                  |
| AST_var   | Assets                | Asset variety (stocks, ETFs, bonds, etc.)  |
| AST_reg   | Assets                | Asset regional/market coverage             |
| LNG       | Language Support      | App language support                       |
| LOC_reg   | Regional Availability | Regional availability & features           |
| PRC_fee   | Pricing               | Fees, spreads, commissions                 |
| PRC_fx    | Pricing               | FX/conversion costs                        |
| SUP_resp  | Support               | Customer support responsiveness            |
| SUP_help  | Support               | Help resources, guides                     |
| GEN       | Overall Satisfaction  | Overall satisfaction                       |
| GEN_trust | Overall Satisfaction  | Trust & brand reliability                  |
| OTH       | Other                 | Catch-all for unclassified aspects         |

### Instructions
1. Extract all aspects mentioned in the review. A review may contain multiple aspects.
2. For each aspect, assign: **aspect_normalized, category, code** (must match schema).
3. Add the **aspect_raw**: the reviewer’s exact wording.
4. Classify sentiment: **Positive**, **Negative**, or **Neutral**.
   - Interpret sentiment correctly not literally, even if irony or sarcasm is present.
   - If the same aspect is mentioned with both positive and negative sentiment, output **separate entries** (not mixed).
5. If no aspects match, return `"aspects": []`.
6. Output must be valid JSON.


Each review must be analyzed separately and returned as:

{
  "review_id": <ID>,
  "aspects": [
    {"aspect_raw": "...", "aspect_normalized": "...", "category": "...", "code": "...", "sentiment": "..."},
    {"aspect_raw": "...", "aspect_normalized": "...", "category": "...", "code": "...", "sentiment": "..."}
    ...
  ]
}

For each aspect:
- aspect_raw: reviewer's wording from the review
- aspect_normalized: Aspect_normalized from schema
- category: Category from schema
- code: Code from schema
- sentiment: Positive or Negative or Neutral

---

### Examples (important!)

**Example 1: Multiple sentiments for same aspect**
Review: “The app is good but crashes often.”

Expected output:
{
  "review_id": 123,
  "aspects": [
    {
      "aspect_raw": "app is good",
      "aspect_normalized": "Mobile app usability",
      "category": "Interface/Usability",
      "code": "INT_app",
      "sentiment": "Positive"
    },
    {
      "aspect_raw": "crashes often",
      "aspect_normalized": "Mobile app usability",
      "category": "Interface/Usability",
      "code": "INT_app",
      "sentiment": "Negative"
    }
  ]
}

**Example 2: Irony detection**
Review: “Amazing support – if you enjoy waiting three days for a reply.”

Expected output:
{
  "review_id": 124,
  "aspects": [
    {
      "aspect_raw": "waiting three days for a reply",
      "aspect_normalized": "Customer support responsiveness",
      "category": "Support",
      "code": "SUP_resp",
      "sentiment": "Negative"
    }
  ]
}

**Example 3: Neutral factual statement**
Review: “The app has a web version.”

Expected output:
{
  "review_id": 125,
  "aspects": [
    {
      "aspect_raw": "has a web version",
      "aspect_normalized": "Web interface usability",
      "category": "Interface/Usability",
      "code": "INT_web",
      "sentiment": "Neutral"
    }
  ]
}

**Example 4: Multiple aspects across categories**
Review: “The fees are low, the design is clean, but support is slow.”

Expected output:
{
  "review_id": 126,
  "aspects": [
    {
      "aspect_raw": "fees are low",
      "aspect_normalized": "Fees, spreads, commissions",
      "category": "Pricing",
      "code": "PRC_fee",
      "sentiment": "Positive"
    },
    {
      "aspect_raw": "design is clean",
      "aspect_normalized": "Design & aesthetics",
      "category": "Interface/Usability",
      "code": "INT_des",
      "sentiment": "Positive"
    },
    {
      "aspect_raw": "support is slow",
      "aspect_normalized": "Customer support responsiveness",
      "category": "Support",
      "code": "SUP_resp",
      "sentiment": "Negative"
    }
  ]
}

**Example 5: No aspects found**
Review: “Downloaded yesterday, will try it later.”

Expected output:
{
  "review_id": 127,
  "aspects": []
}
"""

# ===============================
# 4. API Helper Function
# ===============================
def analyze_review(row):
    """
    Sends a single review to the OpenAI API and returns parsed JSON with review_id.
    """
    user_content = f"Review {row['review_id']}: {row['review']}"

    response = client.chat.completions.create(
        model="gpt-4.1",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": user_content}
        ]
    )

    content = response.choices[0].message.content.strip()

    try:
        parsed = json.loads(content)
        # Ensure review_id is present
        parsed["review_id"] = int(row["review_id"])
        return parsed
    except:
        return {"review_id": int(row["review_id"]), "error": content}

# ===============================
# 5. Batch Processing with Checkpoint
# ===============================
def process_in_batches(df, batch_size=5, checkpoint_path="/content/drive/MyDrive/Colab Notebooks/ABSA-summer-project/absa_results.jsonl"):
    results = []

    # Try resume
    try:
        with open(checkpoint_path, "r") as f:
            for line in f:
                results.append(json.loads(line))
        processed_ids = {r["review_id"] for r in results}
        print(f"Resuming: {len(processed_ids)} already processed")
    except FileNotFoundError:
        processed_ids = set()
        print("Starting fresh")

    # Process
    for i in range(0, len(df), batch_size):
        batch = df.iloc[i:i+batch_size]
        batch = batch[~batch["review_id"].isin(processed_ids)]
        if batch.empty:
            continue

        print(f"Processing {len(batch)} reviews (IDs {batch['review_id'].tolist()})")

        batch_results = []
        for _, row in batch.iterrows():
            try:
                r = analyze_review(row)
                batch_results.append(r)
            except Exception as e:
                r = {"review_id": int(row["review_id"]), "error": str(e)}
                batch_results.append(r)
                time.sleep(5)

        # Save checkpoint
        with open(checkpoint_path, "a") as f:
            for r in batch_results:
                f.write(json.dumps(r) + "\n")

        results.extend(batch_results)
        time.sleep(1)

    return results

# ===============================
# 6. Run ABSA
# ===============================
results = process_in_batches(df, batch_size=8)

# ===============================
# 7. Flatten Results into DataFrame
# ===============================
expanded_rows = []
for r in results:
    if "error" in r:
        continue
    for asp in r["aspects"]:
        expanded_rows.append({
            "review_id": r["review_id"],
            "aspect_raw": asp["aspect_raw"],
            "aspect_normalized": asp["aspect_normalized"],
            "category": asp["category"],
            "code": asp["code"],
            "sentiment": asp["sentiment"]
        })

aspects_df = pd.DataFrame(expanded_rows)
print(aspects_df.head())

# Save final
aspects_df.to_csv("/content/drive/MyDrive/Colab Notebooks/ABSA-summer-project/absa_aspects_flat.csv", index=False)
print("Final results saved to Google Drive")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Total reviews: 1441
Starting fresh
Processing 8 reviews (IDs [1, 2, 3, 4, 5, 6, 7, 8])
Processing 8 reviews (IDs [9, 10, 11, 12, 13, 14, 15, 16])
Processing 8 reviews (IDs [17, 18, 19, 20, 21, 22, 23, 24])
Processing 8 reviews (IDs [25, 26, 27, 28, 29, 30, 31, 32])
Processing 8 reviews (IDs [33, 34, 35, 36, 37, 38, 39, 40])
Processing 8 reviews (IDs [41, 42, 43, 44, 45, 46, 47, 48])
Processing 8 reviews (IDs [49, 50, 51, 52, 53, 54, 55, 56])
Processing 8 reviews (IDs [57, 58, 59, 60, 61, 62, 63, 64])
Processing 8 reviews (IDs [65, 66, 67, 68, 69, 70, 71, 72])
Processing 8 reviews (IDs [73, 74, 75, 76, 77, 78, 79, 80])
Processing 8 reviews (IDs [81, 82, 83, 84, 85, 86, 87, 88])
Processing 8 reviews (IDs [89, 90, 91, 92, 93, 94, 95, 96])
Processing 8 reviews (IDs [97, 98, 99, 100, 101, 102, 103, 104])
Processing 8 reviews (IDs [105, 106, 107, 108, 109, 110, 111